In [1]:
from datasets import load_dataset
from evaluate import load
import pandas as pd
import glob

2023-12-07 16:37:53.368184: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 16:37:54.071191: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-07 16:37:54.071286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-07 16:37:54.071294: W tensorfl

In [2]:
ds = load_dataset("squad_v2")["validation"]
ds

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [3]:
files = "*.pkl"
df = pd.DataFrame()

for file in glob.glob(files):
    read_df = pd.read_pickle(file).fillna(0)
    read_df["id"] = ds[0:120]["id"]
    read_df["answers"] = ds[0:120]["answers"]
    df = pd.concat([df, read_df])

df = df.set_index("name")
df.head()

,question,prediction,is_possible,context,qa_score,correctness_score,reasoning,prompt,id,answers
name,,,,,,,,,,
tiiuae/falcon-7b-instruct,In what country is Normandy located?,Normandy is located in France.,True,The Normans (Norman: Nourmands; French: Norman...,1.0,0.0,ote: This is not asking for your opinion on th...,Answer the following Question based on the Con...,56ddde6b9a695914005b9628,"{'text': ['France', 'France', 'France', 'Franc..."
tiiuae/falcon-7b-instruct,When were the Normans in Normandy?,The Normans were in Normandy from the 10th to ...,True,The Normans (Norman: Nourmands; French: Norman...,0.0,0.0,ote: This is a sample question from an automat...,Answer the following Question based on the Con...,56ddde6b9a695914005b9629,"{'text': ['10th and 11th centuries', 'in the 1..."
tiiuae/falcon-7b-instruct,From which countries did the Norse originate?,"The Normans originated from Denmark, Iceland, ...",True,The Normans (Norman: Nourmands; French: Norman...,0.0,1.0,Do not add any other characters or words.\n\nN...,Answer the following Question based on the Con...,56ddde6b9a695914005b962a,"{'text': ['Denmark, Iceland and Norway', 'Denm..."
tiiuae/falcon-7b-instruct,Who was the Norse leader?,"The Norse leader was Rollo, a Viking who agree...",True,The Normans (Norman: Nourmands; French: Norman...,0.0,0.0,"ote: This is not asking for your opinion, but ...",Answer the following Question based on the Con...,56ddde6b9a695914005b962b,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],..."
tiiuae/falcon-7b-instruct,What century did the Normans first gain their ...,The Normans gained their separate identity in ...,True,The Normans (Norman: Nourmands; French: Norman...,0.0,1.0,Note: This is a sample problem. Your program w...,Answer the following Question based on the Con...,56ddde6b9a695914005b962c,"{'text': ['10th century', 'the first half of t..."


In [7]:
squad_metric = load("squad")

def get_scores(df):
    preds = []
    gold_ref = []
    for (index, row) in df.iterrows():
        preds.append(
            {
               "prediction_text": row.prediction,
                "id": row.id
            }
        )
        gold_ref.append(
            {
                "answers": row.answers if len(row.answers.get("text")) > 0 else {"answer_start": [], "text": ["I don't know."]},
                "id": row.id
            }
        )
    scores = squad_metric.compute(predictions=preds, references=gold_ref)
    return scores.get("f1")

In [8]:
results = []

for model_id in df.index.unique():
    f1_score = get_scores(df.loc[model_id])
    llm_reasoning = df.loc[model_id].correctness_score.sum()
    llm_qa = df.loc[model_id].qa_score.sum()

    results.append(
        {
            "model": model_id,
            "f1": f1_score,
            "llm_reasoning": llm_reasoning,
            "llm_qa": llm_qa
        }
    )

result_df = pd.DataFrame(results)

In [9]:
result_df.set_index("model").sort_values(by="f1", ascending=False)

,f1,llm_reasoning,llm_qa
model,,,
mistralai/Mistral-7B-Instruct-v0.1,38.159354,14.0,4.0
Intel/neural-chat-7b-v3-1,36.154177,14.0,6.0
meta-llama/Llama-2-7b-chat-hf,35.851608,18.0,1.0
tiiuae/falcon-7b-instruct,23.208103,30.0,5.0
microsoft/Orca-2-13b,19.374062,19.0,1.0
HuggingFaceH4/zephyr-7b-beta,12.304035,26.0,10.0
